In [111]:
import torch
import torch.nn as nn
import torch.optim as optim
import clip
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pandas as pd
import pickle as pkl

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.5.1
CUDA available: False
CUDA version: None


In [112]:
def create_conv1d(input_dim=512, kernel_size=3, padding=1, stride=1, num_layers=3, channels_per_layer=[256, 128, 64]):
    print(f"Input size for Conv1D: {input_dim}")
    if len(channels_per_layer) != num_layers:
        raise ValueError("The number of channels per layer must match the number of layers")
    layers = []

    layers.append(nn.Conv1d(in_channels=input_dim, out_channels=channels_per_layer[0], kernel_size=kernel_size, padding=padding, stride=stride))
    layers.append(nn.BatchNorm1d(channels_per_layer[0]))
    layers.append(nn.ReLU())

    in_channels = channels_per_layer[0]
    for out_channels in channels_per_layer[1:]:
        layers.append(nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=padding, stride=stride))
        layers.append(nn.BatchNorm1d(out_channels))
        layers.append(nn.ReLU())
        in_channels = out_channels

    model = nn.Sequential(*layers)
    print(f"Output size for Conv1D: {channels_per_layer[-1]}")  

    return model


def create_mlp(input_dim=532, num_layers=3, hidden_per_layer=[256, 128, 64], output_dim=1):
    print(f"Input size for MLP: {input_dim}")
    if len(hidden_per_layer) != num_layers:
        raise ValueError("The number of hidden units per layer must match the number of layers")
    
    layers = []
    in_features = input_dim
    for out_features in hidden_per_layer:
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())  # Usamos ReLU como función de activación en las capas ocultas
        in_features = out_features
    
    layers.append(nn.Linear(in_features, output_dim))    
    model = nn.Sequential(*layers)
    return model

In [113]:
class CombinedModel(nn.Module):
    def __init__(self, input_dim_images=512, input_dim_tabular=30, kernel_size_images=3, kernel_size_tabular=3, padding=1, stride=1, num_layers_1=3, num_layers_2=3, channels_per_layer_1=[32, 20, 10], channels_per_layer_2=[32, 20, 10], mlp_hidden=[8, 5, 3]):
        super(CombinedModel, self).__init__()
        mlp_input_dim = channels_per_layer_1[-1] + channels_per_layer_2[-1]
        self.conv1 = create_conv1d(input_dim=input_dim_images, kernel_size=kernel_size_images, padding=padding, stride=stride, num_layers=num_layers_1, channels_per_layer=channels_per_layer_1)
        self.conv2 = create_conv1d(input_dim=input_dim_tabular, kernel_size=kernel_size_tabular, padding=padding, stride=stride, num_layers=num_layers_2, channels_per_layer=channels_per_layer_2)
        self.mlp = create_mlp(input_dim=mlp_input_dim, num_layers=len(mlp_hidden), hidden_per_layer=mlp_hidden)

    def forward(self, X_images, X_tabular):
        # Pasar los datos por ambas ramas convolucionales
        x1 = self.conv1(X_images)
        x2 = self.conv2(X_tabular)
        # Concatenar los resultados
        x1 = x1.view(x1.size(0), -1)  # Aplanar
        x2 = x2.view(x2.size(0), -1)  # Aplanar

        x_combined = torch.cat((x1, x2), dim=1)  # Concatenación
        output = self.mlp(x_combined)

        return output

    def train_model(self, X_images_train, X_tabular_train, Y_train, epochs=10, batch_size=32, learning_rate=0.001, loss_fn=F.mse_loss):
        # Initialize optimizer
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        # Convert to TensorDataset and DataLoader
        train_dataset = torch.utils.data.TensorDataset(X_images_train, X_tabular_train, Y_train)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(epochs):
                # Check for NaN or Inf values in the inputs
                if torch.any(torch.isnan(X_images)) or torch.any(torch.isnan(X_tabular)) or torch.any(torch.isnan(y)):
                    print(f"NaN detected in inputs at batch {batch_idx}")
                    continue

                if torch.any(torch.isinf(X_images)) or torch.any(torch.isinf(X_tabular)) or torch.any(torch.isinf(y)):
                    print(f"Inf detected in inputs at batch {batch_idx}")
                    continue

                outputs = self(X_images, X_tabular)
                
                # Check for NaN or Inf in the model's outputs
                if torch.any(torch.isnan(outputs)) or torch.any(torch.isinf(outputs)):
                    print(f"NaN/Inf detected in model output at batch {batch_idx}")
                    continue

                loss = loss_fn(outputs, y)
                print(f"Loss at batch {batch_idx}: {loss.item()}")

                loss.backward()
                optimizer.step()

                running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")


In [114]:
X_tabular_train = pd.read_csv("../data/preprocessed/train/X_train.csv")
y_train = pd.read_csv("../data/preprocessed/train/Y_train.csv")
X_images_train = pkl.load(open("../data/preprocessed/train/train_clip_vectors.pkl", "rb"))


In [115]:
X_tabular_train.head()

,DatePosted,DateTaken,DateCrawl,NumSets,NumGroups,AvgGroupsMemb,AvgGroupPhotos,Tags,Latitude,Longitude,HasStats,Contacts,PhotoCount,MeanViews,GroupsCount,GroupsAvgMembers,GroupsAvgPictures
0,1507147765,1507147765,1507155322,1,0,0.0,0.0,1,NaN,NaN,1,148,7006,21.114331,153,17954.915033,328481.45098
1,1507147800,1507147800,1507151128,1,0,0.0,0.0,1,NaN,NaN,0,55,565,65.538053,4,19875.750000,1215.50000
2,1507147801,1507147801,1507149986,0,0,0.0,0.0,1,NaN,NaN,0,5,562,70.215302,0,0.000000,0.00000
3,1507147811,1507147811,1507154551,1,0,0.0,0.0,1,NaN,NaN,0,485,397,162.307305,0,0.000000,0.00000
4,1507147820,1507147820,1507153563,0,0,0.0,0.0,1,NaN,NaN,0,1,19380,265.643400,0,0.000000,0.00000


In [116]:
y_train.head()

,Day30
0,18.0
1,9.0
2,11.0
3,121.0
4,228.0


In [117]:
#turn df into torch tensor
X_tabular_train = torch.tensor(X_tabular_train.values).float().unsqueeze(2) 
X_images_train = torch.cat(X_images_train, dim=0).unsqueeze(2) 
y_train = torch.tensor(y_train.values).float()



In [118]:
print("X_tabular_shape: ", X_tabular_train.shape)
print("X_images_shape: ", X_images_train.shape)
print("y_shape: ", y_train.shape)


X_tabular_shape:  torch.Size([18303, 17, 1])
X_images_shape:  torch.Size([18303, 512, 1])
y_shape:  torch.Size([18303, 1])


In [119]:

model = CombinedModel(input_dim_images=512, input_dim_tabular=17)
model.train_model(X_images_train, X_tabular_train, y_train, epochs=10, batch_size=32, learning_rate=0.001, loss_fn=F.mse_loss)


Input size for Conv1D: 512
Output size for Conv1D: 10
Input size for Conv1D: 17
Output size for Conv1D: 10
Input size for MLP: 20


NameError: name 'X_images' is not defined